In [1]:
import duckdb

# puzzle input goes here
input = """
"""

In [2]:
duckdb.sql("""
    WITH input AS (
        SELECT '%s' AS input
    ), t AS (
        SELECT row_no, GENERATE_SUBSCRIPTS(STRING_SPLIT(i, ''), 1) col_no, UNNEST(STRING_SPLIT(i, '')) i
        FROM (
            SELECT
                GENERATE_SUBSCRIPTS(STRING_SPLIT(input, '\n'), 1) row_no,
                UNNEST(STRING_SPLIT(input, '\n')) i
            FROM input
        )
    ), empty_rows AS (
        SELECT row_no
        FROM t
        GROUP BY row_no
        HAVING COUNT(*) FILTER(WHERE i <> '.') = 0
    ), empty_cols AS (
        SELECT col_no
        FROM t
        GROUP BY col_no
        HAVING COUNT(*) FILTER(WHERE i <> '.') = 0
    ), expanded_t AS (
        SELECT ROW_NUMBER() OVER() obj_no, row_no, col_no
        FROM (
            SELECT
                i,
                t.row_no + COUNT(er.row_no) OVER(PARTITION BY t.col_no ORDER BY t.row_no) row_no,
                t.col_no + COUNT(ec.col_no) OVER(PARTITION BY t.row_no ORDER BY t.col_no) col_no
            FROM t
            LEFT JOIN empty_rows er
                ON t.row_no = er.row_no
            LEFT JOIN empty_cols ec
                ON t.col_no = ec.col_no
        )
        WHERE i = '#'
    ), pair_dist AS (
        SELECT
            t1.obj_no obj_1,
            t2.obj_no obj_2,
            ABS(t1.row_no - t2.row_no) + ABS(t1.col_no - t2.col_no) dist,
            t1.row_no, t1.col_no, t2.row_no, t2.col_no
        FROM expanded_t t1, expanded_t t2
        WHERE t1 < t2
    )
    SELECT SUM(dist) ans
    FROM pair_dist
""" % input)

┌─────────┐
│   ans   │
│ int128  │
├─────────┤
│ 9591768 │
└─────────┘

In [3]:
duckdb.sql("""
    WITH input AS (
        SELECT '%s' AS input
    ), t AS (
        SELECT row_no, GENERATE_SUBSCRIPTS(STRING_SPLIT(i, ''), 1) col_no, UNNEST(STRING_SPLIT(i, '')) i
        FROM (
            SELECT
                GENERATE_SUBSCRIPTS(STRING_SPLIT(input, '\n'), 1) row_no,
                UNNEST(STRING_SPLIT(input, '\n')) i
            FROM input
        )
    ), empty_rows AS (
        SELECT row_no
        FROM t
        GROUP BY row_no
        HAVING COUNT(*) FILTER(WHERE i <> '.') = 0
    ), empty_cols AS (
        SELECT col_no
        FROM t
        GROUP BY col_no
        HAVING COUNT(*) FILTER(WHERE i <> '.') = 0
    ), expanded_t AS (
        SELECT ROW_NUMBER() OVER() obj_no, row_no, col_no
        FROM (
            SELECT
                i,
                t.row_no + (1000000 - 1) * COUNT(er.row_no) OVER(PARTITION BY t.col_no ORDER BY t.row_no) row_no,
                t.col_no + (1000000 - 1) * COUNT(ec.col_no) OVER(PARTITION BY t.row_no ORDER BY t.col_no) col_no
            FROM t
            LEFT JOIN empty_rows er
                ON t.row_no = er.row_no
            LEFT JOIN empty_cols ec
                ON t.col_no = ec.col_no
        )
        WHERE i = '#'
    ), pair_dist AS (
        SELECT
            t1.obj_no obj_1,
            t2.obj_no obj_2,
            ABS(t1.row_no - t2.row_no) + ABS(t1.col_no - t2.col_no) dist,
            t1.row_no, t1.col_no, t2.row_no, t2.col_no
        FROM expanded_t t1, expanded_t t2
        WHERE t1 < t2
    )
    SELECT SUM(dist) ans
    FROM pair_dist
""" % input)

┌──────────────┐
│     ans      │
│    int128    │
├──────────────┤
│ 746962097860 │
└──────────────┘